In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames[:1]:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from PIL import Image

In [ ]:
img1 = Image.open('/kaggle/input/plant-seedlings-classification/train/Fat Hen/7f731311e.png')

In [ ]:
img1_size = img1.size
print(img1_size)

In [ ]:
import glob

train = pd.DataFrame({'path': glob.glob('/kaggle/input/plant-seedlings-classification/train/*/*')})
train

In [ ]:
train['path'][0].split('/')[-2]

In [ ]:
train['target'] = train['path'].apply(lambda x : x.split('/')[-2])
train

In [ ]:
test = pd.DataFrame({'path': glob.glob('/kaggle/input/plant-seedlings-classification/test/*')})
test

In [ ]:
test['path'][0]

In [ ]:
import tensorflow

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_valid = train_test_split(train, test_size = 0.2, 
                                   random_state = 42, 
                                   stratify = train['target'])

In [ ]:
x_train.shape

In [ ]:
x_valid.shape

In [ ]:
idg = ImageDataGenerator()

train_gen = idg.flow_from_dataframe(x_train, x_col = 'path',
                                   y_col = 'target',
                                   target_size = (100,100))

valid_gen = idg.flow_from_dataframe(x_valid, x_col = 'path',
                                   y_col = 'target',
                                   target_size = (100,100))

test_gen = idg.flow_from_dataframe(test, x_col = 'path',
                                   y_col = None,
                                   class_mode = None,
                                   shuffle = False,
                                   target_size = (100,100))

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.applications.efficientnet import EfficientNetB0

In [ ]:
model = Sequential()

model.add(EfficientNetB0(include_top = False, pooling = 'avg'))
model.add(Dense(12, activation = 'softmax'))

model.compile(loss = 'categorical_crossentropy', optimizer = 'adam',
             metrics = ['acc'])

In [ ]:
model.fit(train_gen, epochs = 3, validation_data = valid_gen )

In [ ]:
result = model.predict(test_gen, verbose = 1)

In [ ]:
result

In [ ]:
result[0]

In [ ]:
sum([9.87884700e-01, 3.90869417e-08, 3.80452647e-08, 4.19852313e-08,
       2.89233640e-06, 1.45056686e-06, 1.20760165e-02, 5.48976296e-08,
       3.11104086e-05, 6.19361273e-09, 1.92199565e-07, 3.40557472e-06])

In [ ]:
sub = pd.read_csv('/kaggle/input/plant-seedlings-classification/sample_submission.csv')

In [ ]:
sub

In [ ]:
y_predict = np.argmax(model.predict(test_gen), axis = -1)

In [ ]:
y_predict

In [ ]:
train_gen.class_indices

In [ ]:
sorted(train['target'].unique())[0]

In [ ]:
class_list = []

for i , prediction in enumerate(y_predict):
    class_list.append(sorted(train['target'].unique())[prediction])

In [ ]:
sub['species'] = class_list
sub